This is the codes that I use for Financial Data Challange PT.Sharing Vision Indonesia x BRI. I won't upload the data, only codes

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv("xxxxxxx.csv")

In [ ]:
pd.set_option('display.max_columns', None)
data.head(15)

In [ ]:
data.info(verbose=True, null_counts=True)

#### I'll skip the Exploratory Data Analysis

In [ ]:
data_prep = data.copy()

# Data split

In [ ]:
from sklearn.model_selection import train_test_split
from collections import Counter

In [ ]:
data_x = data_prep.drop('y', axis=1).copy()
data_y = data_prep['y']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.25, random_state=5)

In [ ]:
print(Counter(y_train))
print(x_train.shape)

# Imputing Missing Value For Categorical

In [ ]:
mod = ['x6', 'x7', 'x21', 'x51', 'x79', 'x80', 'x89', 'x104', 'x108', 'x112', 'x118']

for j in mod:
  x_train[j].fillna(x_train[j].mode().iloc[0], inplace=True)

for j in mod:
  x_test[j].fillna(x_train[j].mode().iloc[0], inplace=True)

# Encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
ohe = OneHotEncoder(categories=[['Laki-laki', 'Perempuan']] ,handle_unknown='ignore', sparse = False)
x_train79 = np.array(x_train['x79']).reshape(-1,1)
ohe.fit(x_train79)
x_train['x79'] = ohe.transform(x_train79)
x_test79 = np.array(x_test['x79']).reshape(-1,1)
x_test['x79'] = ohe.transform(x_test79)

In [ ]:
x_train["x21"] = x_train["x21"].map({"Jan":0,"Feb":1, "Mar":2, "Apr":3, "May":4, "Jun":5,
                                         "Jul":6, "Aug":7, "Sep":8, "Oct":9, "Nov":10, "Dec":11})

x_test["x21"] = x_test["x21"].map({"Jan":0,"Feb":1, "Mar":2, "Apr":3, "May":4, "Jun":5,
                                         "Jul":6, "Aug":7, "Sep":8, "Oct":9, "Nov":10, "Dec":11})

In [ ]:
lenc = LabelEncoder()
lenc.fit(x_train['x89'])
x_train['x89'] = lenc.transform(x_train['x89'])
x_test['x89'] = lenc.transform(x_test['x89'])

In [ ]:
x_train["x112"] = x_train["x112"].map({"Jan":0,"Feb":1, "Mar":2, "Apr":3, "May":4, "Jun":5,
                                         "Jul":6, "Aug":7, "Sep":8, "Oct":9, "Nov":10, "Dec":11})

x_test["x112"] = x_test["x112"].map({"Jan":0,"Feb":1, "Mar":2, "Apr":3, "May":4, "Jun":5,
                                         "Jul":6, "Aug":7, "Sep":8, "Oct":9, "Nov":10, "Dec":11})

In [ ]:
edu = ['Tidak sekolah', 'SD', 'SMP', 'SMA/SMK', 'S1/S2/S3']
ore = OrdinalEncoder(categories=([edu]))
ore.fit(x_train[['x108']])
x_train['x108'] = ore.transform(x_train[['x108']])
x_test['x108'] = ore.transform(x_test[['x108']])

# Imputing Missing Value For Numeric Data
I'm using KNN imputer

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
imputer = KNNImputer(n_neighbors=1, weights='uniform', metric='nan_euclidean')
imputer.fit(x_train)
x_train = imputer.transform(x_train)
x_test = imputer.transform(x_test)

In [ ]:
x_train = pd.DataFrame(x_train, columns=data_x.columns)
x_test = pd.DataFrame(x_test, columns=data_x.columns)

# Feature Selection
I'm using F-score ANOVA for feature selection and used 34 highest F-score

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

In [ ]:
filter = SelectKBest(f_classif, k=34)
filter.fit(x_train, y_train)

x_train_filter = filter.transform(x_train)
x_test_filter = filter.transform(x_test)

print("Before feature selection", x_train.shape)
print("After feature selection", x_train_filter.shape)

In [ ]:
print("Score of features", filter.scores_)
plt.figure(figsize=(20,23))
feature_importance = pd.Series(filter.scores_, index=x_train.columns)
feature_importance.sort_values().plot(kind='barh')
plt.show()

# Modeling

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import plot_confusion_matrix

### I'm using multilayer perceptron from neural network

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
mlp = MLPClassifier(max_iter=120, random_state=5, hidden_layer_sizes=(100,3))

mlp.fit(x_train_filter, y_train)

predictionmlp = mlp.predict(x_test_filter)
print(confusion_matrix(y_test, predictionmlp))
print("Akurasi dari Multilayer Perceptron adalah: %.2f" % (accuracy_score(y_test, predictionmlp)*100) )
print("Recall dari Multilayer Perceptron adalah:",recall_score(y_test, predictionmlp)*100)
print("Precision dari Multilayer Perceptron adalah:",precision_score(y_test, predictionmlp)*100)
print("")

plot_confusion_matrix(mlp, x_test_filter, y_test, cmap=plt.cm.Blues, values_format='.0f')
sns.set(style=None)  
plt.show()

In [ ]:
prediction_test = mlp.predict(x_test_filter)
prediction_train = mlp.predict(x_train_filter)
training_acc = accuracy_score(y_train, prediction_train)
testing_acc = accuracy_score(y_test, prediction_test)

print("Training Accuracy: {}".format(training_acc))
print("Testing Accuracy: {}".format(testing_acc))
print(classification_report(y_test, prediction_test))

## Cross Validation

In [ ]:
kfold = KFold(10, shuffle=True)

scores = cross_validate(MLPClassifier(max_iter=120, random_state=5, hidden_layer_sizes=(100,3)), x_train_filter, y_train, cv=kfold, scoring=['accuracy','precision','recall', 'roc_auc'])

print('Accuracy : ',scores['test_accuracy'].mean())    
print('Precision : ',scores['test_precision'].mean())  
print('Recall/ sensitivity : ',scores['test_recall'].mean()) 
print('AUC ROC : ',scores['test_roc_auc'].mean()) 

## ROC AUC curve

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [ ]:
r_probs = [0 for _ in range(len(y_test))]
mlp_probs = mlp.predict_proba(x_test_filter)

mlp_probs = mlp_probs[:, 1]

r_auc = roc_auc_score(y_test, r_probs)
mlp_auc = roc_auc_score(y_test, mlp_probs)

In [ ]:
print('Random (chance) Prediction: AUROC = %.3f' % (r_auc))
print('MLP: AUROC = %.5f' % (mlp_auc))

In [ ]:
r_fpr, r_tpr, _ = roc_curve(y_test, r_probs)
mlp_fpr, mlp_tpr, _ = roc_curve(y_test, mlp_probs)

In [ ]:
plt.plot(r_fpr, r_tpr, linestyle='--', label='Random prediction (AUROC = %0.3f)' % r_auc)
plt.plot(mlp_fpr, mlp_tpr, marker='.', label='MLP (AUROC = %0.5f)' % mlp_auc)

plt.title('ROC Plot')
# Axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# Show legend
plt.legend() # 
# Show plot
plt.show()

# Prediction

In [ ]:
testing_data = pd.read_csv('xxxxxxxxx.csv')

## Data Split

In [ ]:
data_x = data.drop('y', axis=1)
data_y = data['y']

In [ ]:
mod = ['x6', 'x7', 'x21', 'x51', 'x79', 'x80', 'x89', 'x104', 'x108', 'x112', 'x118']

for j in mod:
  data_x[j].fillna(data_x[j].mode().iloc[0], inplace=True)

for j in mod:
  testing_data[j].fillna(data_x[j].mode().iloc[0], inplace=True)

In [ ]:
ohe = OneHotEncoder(categories=[['Laki-laki', 'Perempuan']] ,handle_unknown='ignore', sparse = False)
data_x79 = np.array(data_x['x79']).reshape(-1,1)
ohe.fit(data_x79)
data_x['x79'] = ohe.transform(data_x79)
testing_datax79 = np.array(testing_data['x79']).reshape(-1,1)
testing_data['x79'] = ohe.transform(testing_datax79)

In [ ]:
data_x["x21"] = data_x["x21"].map({"Jan":0,"Feb":1, "Mar":2, "Apr":3, "May":4, "Jun":5,
                                         "Jul":6, "Aug":7, "Sep":8, "Oct":9, "Nov":10, "Dec":11})

testing_data["x21"] = testing_data["x21"].map({"Jan":0,"Feb":1, "Mar":2, "Apr":3, "May":4, "Jun":5,
                                         "Jul":6, "Aug":7, "Sep":8, "Oct":9, "Nov":10, "Dec":11})

In [ ]:
lenc = LabelEncoder()
lenc.fit(data_x['x89'])
data_x['x89'] = lenc.transform(data_x['x89'])
testing_data['x89'] = lenc.transform(testing_data['x89'])

In [ ]:
data_x["x112"] = data_x["x112"].map({"Jan":0,"Feb":1, "Mar":2, "Apr":3, "May":4, "Jun":5,
                                         "Jul":6, "Aug":7, "Sep":8, "Oct":9, "Nov":10, "Dec":11})

testing_data["x112"] = testing_data["x112"].map({"Jan":0,"Feb":1, "Mar":2, "Apr":3, "May":4, "Jun":5,
                                         "Jul":6, "Aug":7, "Sep":8, "Oct":9, "Nov":10, "Dec":11})

In [ ]:
edu = ['Tidak sekolah', 'SD', 'SMP', 'SMA/SMK', 'S1/S2/S3']
ore = OrdinalEncoder(categories=([edu]))
ore.fit(data_x[['x108']])
data_x['x108'] = ore.transform(data_x[['x108']])
testing_data['x108'] = ore.transform(testing_data[['x108']])

In [ ]:
imputer = KNNImputer(n_neighbors=1, weights='uniform', metric='nan_euclidean')
imputer.fit(data_x)
Xtrain = imputer.transform(data_x)
Xtest = imputer.transform(testing_data)

In [ ]:
Xtrain = pd.DataFrame(Xtrain, columns=data_x.columns)
Xtest = pd.DataFrame(Xtest, columns=testing_data.columns)

In [ ]:
filter = SelectKBest(f_classif, k=34)
filter.fit(Xtrain, data_y)

Xtrain_filter = filter.transform(Xtrain)
Xtest_filter = filter.transform(Xtest)

print("Before feature selection", Xtrain.shape)
print("After feature selection", Xtrain_filter.shape)

In [ ]:
mlp = MLPClassifier(max_iter=120, random_state=5, hidden_layer_sizes=(100,3))

mlp.fit(Xtrain_filter, data_y)

predictionmlp = mlp.predict(Xtest_filter)

In [ ]:
prediction_result = pd.DataFrame(predictionmlp, columns=['Predicted'])

In [ ]:
prediction_result.index.name='Id'

In [ ]:
prediction_result.to_csv('abcdefg.csv', index=True, header=True)